In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

#Resnet Architecture

def conv3x3(in_planes,out_planes,stride=1):
    return nn.Conv2d(in_planes,out_planes,kernel_size=3,stride=stride,padding=1,bias=False)

def conv1x1(in_planes,out_planes,stride=1):
    return nn.Conv2d(in_planes,out_planes,kernel_size=1,stride=stride,bias=False)
###functions are helper functions that create 3x3 and 1x1 convolutional layers, respectively, using the nn.Conv2d class

class BasicBlock(nn.Module): 

    expansion = 1

    def __init__(self,inplanes,planes,stride=1,downsample=None): #planes for output
        super(BasicBlock,self).__init__()
        self.conv1 = conv3x3(inplanes,planes,stride) 
        self.bn1 = nn.BatchNorm2d(planes) #is used to apply batch normalization to the output of the convolutional layers
        self.relu = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(0.5)
        self.conv2 = conv3x3(planes,planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample #is used to match the dimensions of the input tensor to the output tensor of a residual block.
        self.stride = stride

    def forward(self,x):
        identity = x       #It first saves the input tensor x as the identity tensor
        out = self.conv1(x) # then applies the convolutional layers
        out = self.bn1(out) # then  batch normalization layers
        out= self.relu(out) # then ReLU activation function,
        out = self.drop(out) #and dropout layers to the input tensor x to obtain the output tensor out ###prevent overfitting.
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.drop(out)

        if(self.downsample is not None):
            identity = self.downsample(x)
        out += identity 
        out = self.relu(out)

        return out

In [2]:
num_classes = 2
class ResNet(nn.Module):

    def __init__(self,block,layers,num_classes=num_classes):
        super(ResNet,self).__init__()
        self.inplanes = 64 # according to research paper
        self.conv1 = nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size = 3,stride=2,padding=1)
        self.layer1 = self._make_layer(block,64,layers[0],stride=1)
        self.layer2 = self._make_layer(block,128,layers[1],stride=2)
        self.layer3 = self._make_layer(block,256,layers[2],stride=2)
        self.layer4 = self._make_layer(block,512,layers[3],stride=2)
        
        self.avgpooling = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*block.expansion,num_classes)

        for m in self.modules(): 
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,mode="fan_out",nonlinearity="relu") 
            elif isinstance(m,nn.BatchNorm2d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias, 0)


    def _make_layer(self,block,planes,num_layers,stride = 1):
        downsample = None
        if stride!=1 or self.inplanes != planes*block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes,planes*block.expansion,stride),
                nn.BatchNorm2d(planes*block.expansion)
            )
        layers = []
        layers.append(block(self.inplanes,planes,stride,downsample))
        self.inplanes = planes*block.expansion
        for _ in range(1,len(layers)):
            layers.append(block(self.inplanes,planes))

        return nn.Sequential(*layers)

    def forward(self,x):
        x= self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x=self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpooling(x)

        x = x.view(x.size(0),-1) #flatten
        x = self.fc(x)

        return x

In [3]:
model = ResNet(BasicBlock,[2,2,2,2],num_classes=2)

model.load_state_dict(torch.load("300epoch.pth",map_location=torch.device('cpu')))

<All keys matched successfully>